We first mount our Google Drive. The repository of [pytorch-CycleGAN-and-pix2pix](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/) is already cloned under IDP folder. 

In [2]:
import idp_utils.data_handling.constants as C

%cd $C.ROOT_PATH

/home/extra/micheal/IDP


# 1 Install

Install dependencies of 

In [6]:
!cd pytorch-CycleGAN-and-pix2pix && pip install -q -r requirements.txt

# 2 Pretrained models

Download one of the official pretrained models with:

-   `bash ./scripts/download_pix2pix_model.sh [edges2shoes, sat2map, map2sat, facades_label2photo, and day2night]`

Or add your own pretrained model to `./checkpoints/{NAME}_pretrained/latest_net_G.pt`

In [8]:
!bash pytorch-CycleGAN-and-pix2pix/scripts/download_pix2pix_model.sh edges2shoes

Note: available models are edges2shoes, sat2map, map2sat, facades_label2photo, and day2night
Specified [edges2shoes]
for details.

--2022-06-10 13:10:49--  http://efrosgans.eecs.berkeley.edu/pix2pix/models-pytorch/edges2shoes.pth
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.244.190
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.244.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217704688 (208M)
Saving to: ‘./checkpoints/edges2shoes_pretrained/latest_net_G.pth’

./checkpoints/edges 100%[===================>] 207,62M  8,23MB/s    in 29s     

2022-06-10 13:11:19 (7,04 MB/s) - ‘./checkpoints/edges2shoes_pretrained/latest_net_G.pth’ saved [217704688/217704688]



# 3 Training

-   `python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. Add `--direction BtoA` if you want to train a model to transfrom from class B to A.

Before training, use the following command to spin up a Visdom server. Enter [localhost:8097](localhost:8097) to accesss visdom page.

In [ ]:
# !python -m visdom.server -p 8097 > log/visdom.log 2>&1

## 3.1 Train with Original Data

### 3.1.1 AROI

In [ ]:
dataset_folder = C.DATASET_PATTERN.format(data='AROI', name='original')
checkpoint_name = 'pix2pix_aroi_original'
n_epochs = 100
print_freq = 500
batch_size = 64

In [ ]:
!python pytorch-CycleGAN-and-pix2pix/train.py 
        --gpu_ids 0
        --dataroot $dataset_folder \
        --name $checkpoint_name \
        --model pix2pix \
        --direction AtoB \
        --n_epochs $n_epochs \
        --print_freq $print_freq \
        --batch_size $batch_size 

In [ ]:
# python pytorch-CycleGAN-and-pix2pix/train.py \
#         --gpu_ids 1 \
#         --dataroot 'data/datasets/AROI/original' \
#         --name 'pix2pix_aroi_original' \
#         --model pix2pix \
#         --direction AtoB \
#         --n_epochs 100 \
#         --print_freq 500 \
#         --batch_size 64 \
#         > train_pix2pix_aroi_original.log 2>&1 &

### 3.1.2 OP

In [26]:
dataset_folder = C.DATASET_PATTERN.format(data='OP', name='original')
checkpoint_name = 'pix2pix_op_original'
n_epochs = 100
print_freq = 500
batch_size = 64

In [ ]:
!python pytorch-CycleGAN-and-pix2pix/train.py \
        --gpu_ids 0 \
        --dataroot $dataset_folder \
        --name $checkpoint_name \
        --model pix2pix \
        --direction AtoB \
        --n_epochs $n_epochs \
        --print_freq $print_freq \
        --batch_size $batch_size 

----------------- Options ---------------
               batch_size: 64                            	[default: 1]
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: data/datasets/OP/original     	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        
 

## 3.2 Train with Reduced-layer & Original Data

# 4 Testing

-   `python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_pix2pix`

Change the `--dataroot`, `--name`, and `--direction` to be consistent with your trained model's configuration and how you want to transform images.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> Note that we specified --direction BtoA as Facades dataset's A to B direction is photos to labels.

> If you would like to apply a pre-trained model to a collection of input images (rather than image pairs), please use --model test option. See ./scripts/test_single.sh for how to apply a model to Facade label maps (stored in the directory facades/testB).

> See a list of currently available models at ./scripts/download_pix2pix_model.sh

## 4.1 Test with Original Data

### 4.1.1 AROI

In [2]:
dataset_folder = C.DATASET_PATTERN.format(data='AROI', name='original')
dataset_test_folder = os.path.join(dataset_folder, 'test')

In [37]:
!echo $(ls -l $dataset_test_folder |  egrep -c '^-')

377


In [38]:
!NUM_TEST_AROI=$(ls -l $dataset_test_folder |  egrep -c '^-')

In [3]:
dataset_folder = C.DATASET_PATTERN.format(data='AROI', name='original')
checkpoint_name = 'pix2pix_aroi_original'

!python pytorch-CycleGAN-and-pix2pix/test.py \
        --dataroot $dataset_folder \
        --direction AtoB \
        --name $checkpoint_name \
        --model pix2pix \
        --num_test 377

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: data/datasets/AROI/original   	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            

### 4.1.2 OP

In [ ]:
dataset_folder = C.DATASET_PATTERN.format(data='OP', name='original')
dataset_test_folder = os.path.join(dataset_folder, 'test')

In [ ]:
! ls -l $dataset_test_folder | egrep -c '^-'

5355


In [ ]:
dataset_folder = C.DATASET_PATTERN.format(data='OP', name='original')
checkpoint_name = 'pix2pix_op_original'

!python pytorch-CycleGAN-and-pix2pix/test.py \
        --dataroot $dataset_folder \
        --direction AtoB \
        --name $checkpoint_name \
        --model pix2pix \
        --num_test 5355

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: data/datasets/OP/original     	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            